# Hello world

In [30]:
import os
import openai

def GPT_Completion(texts):## Call the API key under your account (in a secure way)
    openai.api_key = "***REMOVED***"

    response = openai.Completion.create(engine="text-davinci-002",
                                        prompt =  texts,
                                        temperature = 0.6,
                                        top_p = 1,
                                        max_tokens = 64,
                                        frequency_penalty = 0,
                                        presence_penalty = 0)
    return print(response.choices[0].text)

recipe = 'Provide a cooking recipe based on the following ingredients: \\n \nApple \
          \n \nFlour \
          \n \nChicken \
          \n \nSalt'

GPT_Completion(recipe)



Apple Chicken:

1. Preheat oven to 375 degrees.
2. Coat chicken in flour.
3. Place chicken in baking dish.
4. Cut up apples into thin slices and place on top of chicken.
5. Sprinkle with salt.
6. Bake for 30-


# Generate training dataset

## Generate problems

In [3]:
import os
import filecmp

## cleaning work directories 
!rm dataset/problems/\* &> /dev/null
!rm dataset/plans/\* &> /dev/null
!rm dataset/plans_ipc/\* &> /dev/null
!rm dataset/formatted/\* &> /dev/null

## 8000 data points should improve performance between 2x and 4x
%run generator/pddl_confgen.py 4 8000 -r 24 -co
!mv -v generator/problems/conditional/* dataset/problems > /dev/null

## clean duplicates

problems = sorted(os.listdir('./dataset/problems/'))

duplicates = 0
to_be_removed = []

for i, problem_i in enumerate(problems):
    print("Cleaning --- " + str(round((i / len(problems) * 100), 1)) +"%", end = '\r')
    for problem_j in problems[i + 1:]:
        if filecmp.cmp('./dataset/problems/' + problem_i, 
                       './dataset/problems/' + problem_j, 
                       shallow=False):
            duplicates += 1
            to_be_removed.append(problem_j)
            
for problem in to_be_removed:
    try:
        os.remove('./dataset/problems/' + problem)
    except FileNotFoundError:
        continue
            
print(str(duplicates) + " duplicates deleted")

Writing conditional problem files to /home/alessio/planning/teriyaki/generator/problems/conditional...
Done.
All operations completed. Happy planning!
24 duplicates deleted


## Generate plans with Probe

In [18]:
import os
import subprocess

problems = sorted(os.listdir('./dataset/problems/'))


for i, problem in enumerate(problems):
    
    result = subprocess.run(['./generator/probe', 
                             '-d', './dataset/domains/domain_m_25_B.pddl', 
                             '-i', './dataset/problems/' + problem,
                             '-o', './dataset/plans/m_25_B-' + str(problem.split('_',2)[2:][0])], 
                             stdout=subprocess.PIPE)
    print("Planning --- " + str(round((i / len(problems) * 100), 1)) +"%", end = '\r')
    
    # temporarly limit to 400 samples
    if i > 500: break

print("Planning complete")

terminate called after throwing an instance of 'std::bad_alloc'
  what():  std::bad_alloc


Planning complete


## Convert Probe plans to IPC format

In [7]:
import os

plans = sorted(os.listdir('./dataset/plans/'))

for plan in plans:
    probe = open('./dataset/plans/' + plan, "r")
    val = open('./dataset/plans_ipc/' + plan.split(".")[0] + ".plan", 'w+')
    replacement = ""
    # using the for loop
    action_timing = 1
    for line in probe:
        line = line.lower()
        line = '%.3f' % (action_timing / 1000) + "00: " + line
        val.write(line)
        action_timing += 2

    probe.close()
    val.close()
    
print("Plans converted to IPC format")

Plans converted to IPC format


## Validate and remove invalid plans using VAL Validate
This step takes a lot of time and usually all plans are validated. **It can be skipped**.

In [8]:
import os
import subprocess

plans = sorted(os.listdir('./dataset/plans_ipc/'))
invalid_plans = 0

for i, plan in enumerate(plans):
    result = subprocess.run(['./generator/Validate', 
                             './dataset/domains/domain_m_25_B.pddl', 
                             './dataset/problems/problem_conditional_' + plan.split("-")[1].split(".")[0] + ".pddl",
                             './dataset/plans_ipc/' + plan], 
                             stdout=subprocess.PIPE)
    if "Plan failed to execute" in str(result):
        invalid_plans += 1
#         os.remove('./dataset/plans_ipc/m_25_B-' + problem.split('_',2)[2:][0].split(".")[0] + ".plan")
#         os.remove('./dataset/problems/' + problem)

    print("Validating --- " + str(round((i / 8000 * 100), 1)) +"%", end = '\r')

print('%i plans could not be validated and have been removed from the dataset' % invalid_plans)

0 plans could not be validated and have been removed from the dataset


## Compile GPT3 training dataset
TODO: `joint3)\n)\n)\n` is seen as part of the prompt

From the documentation regarding conditional generation:
* Use a separator at the end of the prompt, e.g. \n\n###\n\n. Remember to also append this separator when you eventually make requests to your model.
* Use an ending token at the end of the completion, e.g. END
* Remember to add the ending token as a stop sequence during inference, e.g. stop=[" END"]

Example:
```{"prompt":"<Product Name>\n<Wikipedia description>\n\n###\n\n", "completion":" <engaging ad> END"}```

In [21]:
import os
import subprocess

dataset = open('./dataset/formatted/teriyaki.jsonl', 'w+')
plans = sorted(os.listdir('./dataset/plans_ipc/'))

# extracting prompts from problems and completions from IPC plans 
for i, plan in enumerate(plans):
    plan_f = open('./dataset/plans_ipc/' + plan, "r")
    problem_f = open('./dataset/problems/problem_conditional_' + plan.split("-")[1].split(".")[0] + ".pddl", "r")
    lines = problem_f.readlines()
    
    # lines[49:62] is the range of meaningful predicates at initial state
    dataset.write(repr('{"prompt":"\n(:init\n' + ''.join(lines[49:-1]).replace("    ", "") + 
          '\n\n###\n\n", "completion":" ' + plan_f.read().replace("    ", "") + ' END"}')[1:-1] + "\n")

dataset.close()

print("Dataset ready!")
print("Before training, run in a terminal\n\t>openai tools fine_tunes.prepare_data -f './dataset/formatted/teriyaki.jsonl'")

Dataset ready!
Before training, run in a terminal
	>openai tools fine_tunes.prepare_data -f './dataset/formatted/teriyaki.jsonl'


## Compute training cost
`1 token ~= 4 chars
0.0300$/1K Tokens`

In practice it is ~15$ per 1000 samples for this specific planning domain

In [22]:
import os

dataset = open('./dataset/formatted/teriyaki.jsonl', 'r')
chars = len(dataset.read())
dataset.close()
tokens = chars / 4
cost = tokens * 0.00003

print("Training this dataset will consume about {0} tokens for an estimated cost of {1}$".format(tokens, round(cost,2)))

Training this dataset will consume about 164945.75 tokens for an estimated cost of 4.95$


## Creating a customized model and training!
Model: **ft-dVpnKEC46BV9odOlRsGLO7nd** - **davinci:ft-personal-2022-08-18-17-42-22**

From the documentation regarding conditional generation:
* Aim for at least ~500 examples
* Ensure that the prompt + completion doesn't exceed 2048 tokens, including the separator
* Ensure the examples are of high quality and follow the same desired format
* Ensure that the dataset used for finetuning is very similar in structure and type of task as what the model will be used for
* Using Lower learning rate and only 1-2 epochs tends to work better for these use cases

In [36]:
import os
import openai

!openai -k ***REMOVED*** \
api fine_tunes.create -t './dataset/formatted/teriyaki.jsonl' -m 'davinci' --n_epochs 1

Upload progress: 100%|██████████████████████| 660k/660k [00:00<00:00, 1.02Git/s]
Uploaded file from ./dataset/formatted/teriyaki.jsonl: file-IvVN6w5cZHMNmx6i5o5FHBs3
Created fine-tune: ft-dVpnKEC46BV9odOlRsGLO7nd
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-08-18 19:33:20] Created fine-tune: ft-dVpnKEC46BV9odOlRsGLO7nd
[2022-08-18 19:33:30] Fine-tune costs $7.00
[2022-08-18 19:33:31] Fine-tune enqueued. Queue number: 0
[2022-08-18 19:33:33] Fine-tune started
[2022-08-18 19:41:43] Completed epoch 1/1
[2022-08-18 19:42:22] Uploaded model: davinci:ft-personal-2022-08-18-17-42-22
[2022-08-18 19:42:24] Uploaded result file: file-mfdWBbs4HOcYtTf3WJoLCtI5
[2022-08-18 19:42:24] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-personal-2022-08-18-17-42-22 -p <YOUR_PROMPT>


## Test it!

In [9]:
# !openai -k ***REMOVED*** \
# api completions.create -m 'davinci:ft-personal-2022-08-18-17-42-22' \
# -p '(:init\n(angle_joint angle0 joint1)\n(angle_joint angle345 joint2)\n(angle_joint angle0 joint3)\n(in-centre joint3)\n(in-hand link1)\n(in-hand link2)\n(grasp gleft link1)\n(grasp gright link2))\n(:goal(and\n(angle_joint angle285 joint1)\n(angle_joint angle285 joint2)\n(angle_joint angle315 joint3)\n)\n)\n)\n\n\n###\n\n'

import os
import openai

def GPT_Completion(texts):## Call the API key under your account (in a secure way)
    openai.api_key = "***REMOVED***"

    response = openai.Completion.create(engine="davinci:ft-personal-2022-08-18-17-42-22",
                                        prompt =  texts,
                                        top_p = 1,       # modify ONLY top_p or temperature, not both       
                                        temperature = 0, # argmax
                                        max_tokens = 400,
                                        presence_penalty = 0, # -2 < x < 2, negative favors repetitions
                                        frequency_penalty = 0,
                                        best_of = 4, # 3-5 work well for code completion
                                        stop = "END")
    return print(response.choices[0].text)

problem = '(:init\n(angle_joint angle0 joint1)\n(angle_joint angle345 joint2)\n(angle_joint angle0 joint3)\n(in-centre joint3)\n(in-hand link1)\n(in-hand link2)\n(grasp gleft link1)\n(grasp gright link2))\n(:goal(and\n(angle_joint angle285 joint1)\n(angle_joint angle285 joint2)\n(angle_joint angle315 joint3)\n)\n)\n)\n\n\n###\n\n'

GPT_Completion(problem)

 0.00100: (release-links link2 link1 joint1 gleft gright)
0.00200: (link-to-central-grasp link2 link1 joint3 joint1 gleft gright)
0.00300: (decrease_angle_first_child link2 link1 joint1 angle0 angle345 gleft gright)
0.00400: (decrease_angle_first_child link2 link1 joint1 angle345 angle330 gleft gright)
0.00500: (decrease_angle_first_child link2 link1 joint1 angle330 angle315 gleft gright)
0.00600: (release-links link2 link1 joint1 gleft gright)
0.00700: (link-to-central-grasp link3 link2 joint1 joint2 gleft gright)
0.00800: (increase_angle_first_child_45 link3 link2 joint2 angle270 angle285 angle300 angle315 gleft gright)
0.00900: (release-links link3 link2 joint2 gleft gright)
0.01000: (link-to-central-grasp link4 link3 joint2 joint3 gleft gright)
0.01100: (decrease_angle_first_child link4 link3 joint3 angle345 angle330 gleft gright)
0.01200: (decrease_angle_first_child link4 link3 joint3 angle330 angle315 gleft gright)
 


## Run extended tests on the dataset
Plan with Teriyaki for all available problems and run VAL Validate

It reports:
* success rate
* min, max, avg and standard deviation of planning times
* average number of actions before Teryiaki makes a mistake

## Iterative planning trick
* Try to plan only N actions 
* Use VAL **ValStep** to simulate the outcomes of said actions and save it to a problem file
* Replan using the new problem file
* Iterate until goal state is reached